In [1]:
import sys
sys.path.append('../scripts/')
from utility import read_file, create_folder, temp_record_query, temp_record_sdf

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = (
    # Create a spark session (which will run spark jobs)
    SparkSession.builder.appName("Project 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config('spark.executor.memory','10g')
    .config('spark.driver.memory','12g')
    .config('spark.driver.maxResultsSize', '10 GiB')
    .config('spark.shuffle.file.buffer', '64k')
    # .config("spark.network.timeout", "3600s")
    # .master("local[6]")
    .getOrCreate()
    )

22/09/19 12:40:40 WARN Utils: Your hostname, SukiXuudeMacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 100.70.12.31 instead (on interface en0)
22/09/19 12:40:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/19 12:40:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/09/19 12:40:42 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/09/19 12:40:42 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


### 1. Connect Merchants and Transactions AS `sdf`

In [4]:
print("================ Merchant ================")
merchants_sdf = read_file(spark, 'merchants_data.parquet', '../data/curated/', truncate=20)
temp = merchants_sdf.dropna(how='any').count()
print( f'There are {merchants_sdf.count()} data, and {temp} of them have no null value.')

================ Merchant ================
|> Loading File...


|> Loading Finished!
-RECORD 0-----------------------------
 merchant_name | Felis Limited        
 tags          | furniture, home f... 
 merchant_abn  | 10023283211          
 take_rate     | 0.18                 
 type          | e                    
only showing top 1 row

There are 4026 data, and 4026 of them have no null value.


In [5]:
print("================ Transactions ================")
transactions_sdf = read_file(spark, 'transactions_20210228_20210827_snapshot')
temp = transactions_sdf.dropna(how='any').count()
print( f'There are {transactions_sdf.count()} data, and {temp} of them have no null value.')

================ Transactions ================
|> Loading File...


|> Loading Finished!
-RECORD 0----------------------------------------------
 user_id        | 18478                                
 merchant_abn   | 62191208634                          
 dollar_value   | 63.255848959735246                   
 order_id       | 949a63c8-29f7-4ab0-ada4-99ac50a88952 
 order_datetime | 2021-08-20                           
only showing top 1 row



There are 3643266 data, and 3643266 of them have no null value.


- Known that Transactions and Merchants table have no null value

In [6]:
sdf = transactions_sdf.join(merchants_sdf, how='left', on='merchant_abn')
temp = sdf.dropna(how='any', subset=['merchant_name']).count()
print( f'There are {sdf.count()} data, and {temp} of them have no null value.')

There are 3643266 data, and 3494038 of them have no null value.


- After connect the Transactions and Merchants Table by `merchant_abn`, we find that some Transactions not from the merchants in the table. Because they are not in our list of rank therefore, delete those null value.

In [7]:
# sdf = sdf.dropna(how='any', subset=['merchant_name'])

- There are 4026 merchants, and for `transactions_20210228_20210827_snapshot` have 3643266 records. 

    After combination, there are **3494038** rest. **3643266 - 3494038 = 149228** records has been deleted. 
    
    `149228 / 3643266 = 0.04095995186736297`

In [8]:
sdf.show(5)

+------------+-------+------------------+--------------------+--------------+--------------------+--------------------+---------+----+
|merchant_abn|user_id|      dollar_value|            order_id|order_datetime|       merchant_name|                tags|take_rate|type|
+------------+-------+------------------+--------------------+--------------+--------------------+--------------------+---------+----+
| 62191208634|  18478|63.255848959735246|949a63c8-29f7-4ab...|    2021-08-20|Cursus Non Egesta...|furniture, home f...|     2.17|   c|
| 15549624934|      2| 130.3505283105634|6a84c3cf-612a-457...|    2021-08-20|  Commodo Associates|opticians, optica...|     2.76|   c|
| 64403598239|  18479|120.15860593212783|b10dcc33-e53f-425...|    2021-08-20|Lobortis Ultrices...|music shops - mus...|     6.31|   a|
| 60956456424|      3| 136.6785200286976|0f09c5a5-784e-447...|    2021-08-20|Ultricies Digniss...|gift, card, novel...|     4.69|   b|
| 94493496784|  18479| 72.96316578355305|f6c78c1a-4600-

### 2. `sdf` join with Customer

In [9]:
print("================ Customer ================")
customer_sdf = read_file(spark, 'new_consumer_data.parquet', '../data/curated/')
temp = customer_sdf.dropna(how='any').count()
print( f'There are {customer_sdf.count()} data, and {temp} of them have no null value.')

================ Customer ================
|> Loading File...
|> Loading Finished!
-RECORD 0---------------------------------
 consumer_id | 680810                     
 name        | Courtney Mendez            
 address     | 802 Knapp Harbors Apt. 769 
 state       | QLD                        
 postcode    | 9013                       
 gender      | Female                     
 user_id     | 71144                      
only showing top 1 row

There are 499999 data, and 499999 of them have no null value.


In [10]:
sdf = sdf.join(customer_sdf, how='left', on='user_id')
temp = sdf.dropna(how='any', subset=['consumer_id']).count()
print( f'There are {sdf.count()} data, and {temp} of them have no null value.')

There are 3643266 data, and 3643266 of them have no null value.


In [11]:
sdf.show(5)

+-------+------------+------------------+--------------------+--------------+--------------------+--------------------+---------+----+-----------+------------+--------------------+-----+--------+-----------+
|user_id|merchant_abn|      dollar_value|            order_id|order_datetime|       merchant_name|                tags|take_rate|type|consumer_id|        name|             address|state|postcode|     gender|
+-------+------------+------------------+--------------------+--------------+--------------------+--------------------+---------+----+-----------+------------+--------------------+-----+--------+-----------+
|      7| 96442520325|29.863817960377617|48dd50dd-291d-43d...|    2021-04-07|Nullam Lobortis Q...|computers, comput...|     3.65|   b|     511685|Andrea Jones|   122 Brandon Cliff|  QLD|    4606|     Female|
|      7| 29566626791| 80.43460573195489|13d12435-4f1d-4b6...|    2021-04-07|                null|                null|     null|null|     511685|Andrea Jones|   122 Br

### 3. `sdf` join with the external dataset  --- rrm

In [11]:
print("================ RRM_External ================")
rrm_sdf = read_file(spark, 'external_rrm.parquet', '../data/curated/', truncate=20)
temp = rrm_sdf.dropna(how='any').count()
print( f'There are {rrm_sdf.count()} data, and {temp} of them have no null value.')

================ RRM_External ================
|> Loading File...
|> Loading Finished!


-RECORD 0----------
 Date | 2020-02-21 
 AUS  | 1          
 NSW  | 1          
 VIC  | 1          
 QLD  | -1         
 SA   | 2          
 WA   | 2          
 TAS  | -1         
 NT   | 4          
 ACT  | 4          
only showing top 1 row

There are 761 data, and 761 of them have no null value.


In [12]:
sdf = sdf.join(rrm_sdf, sdf.order_datetime == rrm_sdf.Date, 'left')
temp = sdf.dropna(how='any', subset=['Date']).count()
print( f'There are {sdf.count()} data, and {temp} of them have no null value.')

There are 3643266 data, and 3643266 of them have no null value.


In [13]:
sdf.show(5)

+-------+------------+------------------+--------------------+--------------+--------------------+--------------------+---------+----+-----------+------------+--------------------+-----+--------+-----------+----------+---+---+---+---+---+---+---+---+---+
|user_id|merchant_abn|      dollar_value|            order_id|order_datetime|       merchant_name|                tags|take_rate|type|consumer_id|        name|             address|state|postcode|     gender|      Date|AUS|NSW|VIC|QLD| SA| WA|TAS| NT|ACT|
+-------+------------+------------------+--------------------+--------------+--------------------+--------------------+---------+----+-----------+------------+--------------------+-----+--------+-----------+----------+---+---+---+---+---+---+---+---+---+
|      7| 96442520325|29.863817960377617|48dd50dd-291d-43d...|    2021-04-07|Nullam Lobortis Q...|computers, comput...|     3.65|   b|     511685|Andrea Jones|   122 Brandon Cliff|  QLD|    4606|     Female|2021-04-07|-12|-11|-17|-16|-

In [14]:
sdf = sdf.drop('Date')

- 处理 external table 的信息，只保留其所对应的洲的数值

In [15]:
import pandas as pd

In [16]:
cols = ['state', 'AUS', 'NSW', 'VIC', 'QLD', 'SA', 'WA', 'TAS', 'NT', 'ACT']
temp_df = sdf.select(['order_id'] + cols).toDF('order_id', 'state', 'AUS', 'NSW', 'VIC', 'QLD', 'SA', 'WA', 'TAS', 'NT', 'ACT')
temp_df = pd.DataFrame(temp_df.collect(), columns=['order_id'] + cols)

In [17]:
temp_df['target'] = temp_df.apply(lambda x: x[x['state']], axis=1)
temp_df = temp_df.drop(cols, axis=1)

In [18]:
temp_sdf = spark.createDataFrame(temp_df)
temp_sdf.show(5)

22/09/19 12:43:41 WARN TaskSetManager: Stage 79 contains a task of very large size (19939 KiB). The maximum recommended task size is 1000 KiB.


22/09/19 12:43:46 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 79 (TID 367): Attempting to kill Python Worker
+--------------------+------+
|            order_id|target|
+--------------------+------+
|949a63c8-29f7-4ab...|    -6|
|6a84c3cf-612a-457...|   -47|
|b10dcc33-e53f-425...|    -6|
|0f09c5a5-784e-447...|    11|
|f6c78c1a-4600-4c5...|    -6|
+--------------------+------+
only showing top 5 rows



In [19]:
sdf = sdf.join(temp_sdf, on='order_id', how='left')

In [20]:
sdf.show(5)

22/09/19 12:43:48 WARN TaskSetManager: Stage 83 contains a task of very large size (19939 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+-------+------------+------------------+--------------+--------------------+--------------------+---------+----+-----------+------------+--------------------+-----+--------+-----------+---+---+---+---+---+---+---+---+---+------+
|            order_id|user_id|merchant_abn|      dollar_value|order_datetime|       merchant_name|                tags|take_rate|type|consumer_id|        name|             address|state|postcode|     gender|AUS|NSW|VIC|QLD| SA| WA|TAS| NT|ACT|target|
+--------------------+-------+------------+------------------+--------------+--------------------+--------------------+---------+----+-----------+------------+--------------------+-----+--------+-----------+---+---+---+---+---+---+---+---+---+------+
|3dba1ecb-1d9e-415...|  18482| 47529937246|110.27705449024003|    2021-05-24|Nisi Dictum Augue...|books, periodical...|     2.85|   c|     918448|  Ethan Webb|94306 Ray Springs...|  NSW|    1430|       Male| -5| -7|-13|  1|  1|  5| -6| 26|  2|    

### 4. `sdf` join with the external dataset  --- ncd

In [21]:
print("================  ================")
ncd_sdf = read_file(spark, 'external_ncd.parquet', '../data/curated/')
temp = ncd_sdf.dropna(how='any').count()
print( f'There are {ncd_sdf.count()} data, and {temp} have no null value.')

================  ================
|> Loading File...
|> Loading Finished!
-RECORD 0---------------------
 Date            | 2020-10-23 
 Net cases / day | 18         
only showing top 1 row

There are 687 data, and 687 have no null value.


In [22]:
ncd_sdf.show(5)

+----------+---------------+
|      Date|Net cases / day|
+----------+---------------+
|2020-10-23|             18|
|2020-10-24|             15|
|2020-10-25|             21|
|2020-10-26|              7|
|2020-10-27|             14|
+----------+---------------+
only showing top 5 rows



In [23]:
sdf = sdf.join(ncd_sdf, sdf.order_datetime == ncd_sdf.Date, 'left')
sdf.show(5)

22/09/19 12:43:55 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
22/09/19 12:43:55 WARN TaskSetManager: Stage 110 contains a task of very large size (19939 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+-------+------------+------------------+--------------+--------------------+--------------------+---------+----+-----------+------------+--------------------+-----+--------+-----------+---+---+---+---+---+---+---+---+---+------+----------+---------------+
|            order_id|user_id|merchant_abn|      dollar_value|order_datetime|       merchant_name|                tags|take_rate|type|consumer_id|        name|             address|state|postcode|     gender|AUS|NSW|VIC|QLD| SA| WA|TAS| NT|ACT|target|      Date|Net cases / day|
+--------------------+-------+------------+------------------+--------------+--------------------+--------------------+---------+----+-----------+------------+--------------------+-----+--------+-----------+---+---+---+---+---+---+---+---+---+------+----------+---------------+
|3dba1ecb-1d9e-415...|  18482| 47529937246|110.27705449024003|    2021-05-24|Nisi Dictum Augue...|books, periodical...|     2.85|   c|     918448|  Ethan Webb|94306 R

In [24]:
temp = sdf.dropna(how='any', subset=['Date']).count()
print( f'There are {sdf.count()} data, and {temp} have no null value.')

22/09/19 12:44:45 WARN TaskSetManager: Stage 129 contains a task of very large size (19939 KiB). The maximum recommended task size is 1000 KiB.


22/09/19 12:44:53 WARN TaskSetManager: Stage 139 contains a task of very large size (19939 KiB). The maximum recommended task size is 1000 KiB.


There are 3660963 data, and 3643975 have no null value.


In [28]:
sdf = sdf.drop('Date')

In [27]:
# temp = sdf.to_pandas_on_spark()
# temp[temp.Date.isna()]


In [29]:
temp_record_sdf(sdf, path='../data/curated/dataset.parquet', overwrite=True)

|> Waitting for saving...


22/09/19 12:46:01 WARN TaskSetManager: Stage 176 contains a task of very large size (19939 KiB). The maximum recommended task size is 1000 KiB.


|> Save Successfully!


### 5. `sdf` join with the fraud datasets